In [ ]:
#here, this will rewrite bank_accounts_ids to make sure people, companies and banks are in sync

In [87]:
import pandas as pd
import numpy as np
import string
import random 
import requests
from datetime import datetime
 


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

countries = pd.read_excel('countries.xlsx')
people  = pd.read_excel( 'people.xlsx' )   


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
countries = pd.read_excel('countries.xlsx')
companies= pd.read_excel( 'companies.xlsx')  
bank_accounts_ids  = pd.read_excel( 'bank_accounts_ids.xlsx' )   
banks=  pd.read_excel( 'banks.xlsx' )  
print('done')

done


In [88]:
#functions:  

 
#return a unique idn, you can mention where to search 
#you can decide the nr_digits and nr_letters in the idn
def unique_idn(nr_digits=10, nr_letters=0, search_list=[]):
#         idn= "".join(list (map(str, list( np.random.randint(0,10,nr_digits)))))
        letters =list ( np.random.choice(list (string.ascii_letters), size=nr_letters, replace=True))
        digits = list (np.random.choice(list( string.digits), size=nr_digits  ))
        idn=letters + digits


        random.shuffle(idn)
        idn="".join (idn) 
        
    #if exists: return the function again
        if idn in search_list :
           #print('not unique')
           return unique_idn(nr_digits,search_list)
        return idn
unique_idn(nr_digits=4,nr_letters=3)

#takes a string, return a list of corresponding numbers, every charachter is mapped to a number. 
#This is necessary to conclude the check_digits for every iban
def convert_to_numbers( strng):
    strng=strng.lower()
    # a dict that maps each letter to a number
    letter_to_number = {letter: index + 10 for index, letter in enumerate(string.ascii_lowercase)}
    list_numbers=[]
    for c in strng:
        if c.isdigit():
            list_numbers.append(c)
        else:
            list_numbers.append(str (letter_to_number[c]))
    return list_numbers
    
 
convert_to_numbers('60161331926819GB29')





# return an iban ( also acc_nr) 



def generate_new_bank_acc(country_code='NL',bank_code='', location_code='' , acc_nr=''): 
          #genertate an account number: random 12 digits 
        acc_nr= "".join(list (map(str, list( np.random.randint(0,12,10)))))
         
         

        
            #the iban is CCkk bbbb ssss cccc cccc cccc where:
        #     CC: Country Code (2 letters)
        # kk: Check Digits (2 digits)
        # bbbb: Bank Code (varies by country)  
        # ssss: Branch Code (optional, varies by country)
        # cccc cccc cccc: Account Number (the individual bank account number)
        
        #for the bbbb, we will get it from the banks dataset, or you can generate a random number of 4 digits


        #first form the strng of the IBAN : <<to do >> check the form of the list
        strng = f'{location_code}{acc_nr}{bank_code}{country_code}'
        #then calculate the check code : 
          #first convert to one big number
        big_nr= int("".join (convert_to_numbers(strng))  )  
          #Perform Modulo 97 Calculation : 98 - ( remainder when deviding by 97)
        remainder97= big_nr%97
        check_digits =98- remainder97
        iban = f'{country_code}{check_digits}{bank_code}{location_code}{acc_nr}'
        currency = countries.query("country_code==@country_code").currency.values[0]

     #  <<TODO:  make sure it is unique >>: search for that iban in the bank_accounts_ids df, 
    #if exists: return the function again
        if iban in bank_accounts_ids.iban.tolist():
          # print('not unique')
           return generate_new_bank_acc(country_code=country_code,bank_idf='', location_code='' , acc_nr='')

       # some ibans have less charachtaristics than others!
   # <<TO DO: fix this problem>>
        
    ##  
        return {'iban':iban, 
               'acc_nr':acc_nr,
               'country_code':country_code,
               'currency':currency}  
    
# person= generate_person()
generate_new_bank_acc()    
# generate_iban(country_code,bank_idf, location_code , acc_nr)

{'iban': 'NL391111181090107',
 'acc_nr': '1111181090107',
 'country_code': 'NL',
 'currency': 'EUR'}

In [127]:
#0- truncating bank_accounts_ids
bank_accounts_ids   = bank_accounts_ids.iloc[0:0]


In [128]:
#1 banks ready:
#loop over banks, generate idn	bank_acc	iban	balance

for index, row in banks.iterrows():
#     idn=row['idn']
#     iban = row['iban'] 
    
#     if int(idn) not in   bank_accounts_ids.idn.tolist() and iban not in bank_accounts_ids.iban.tolist():

    country_code= row['country_code']
    country= row['country'] 
    bank_code= row['bank_code']
    idn= unique_idn(nr_digits=10, search_list=bank_accounts_ids.idn.tolist()) 
    total_capacity = row['total_capacity']
    balance= total_capacity*1000000 # make it more realistic later?



    bank_acc= generate_new_bank_acc(country_code=country_code,bank_code=bank_code, location_code='' , acc_nr='')
    #now write to the df
    banks.loc[index, 'idn'] = idn
    banks.loc[index, 'iban'] =  bank_acc['iban'] 
    banks.loc[index, 'acc_nr'] = bank_acc['acc_nr']
    banks.loc[index, 'balance'] = balance

    #now write to bank_accounts_ids
    bank_accounts_ids_new_dict = {'idn':idn,  'iban':	bank_acc['iban'],

                                      'acc_nr':bank_acc['acc_nr']  ,
                                      'currency': bank_acc['currency'],
                                      'country_code': country_code,
                                      'country':country,
                                      'balance': balance ,
                                  'bank_name':  row['company'],
                                     'bank_idn':idn}

    bank_accounts_ids_new  = pd.DataFrame([bank_accounts_ids_new_dict])
#         print ( bank_accounts_ids_new ) 

    bank_accounts_ids  = pd.concat([bank_accounts_ids,bank_accounts_ids_new  ], ignore_index=True)  

print('all banks are assigned accounts')    
 

all banks are assigned accounts


In [52]:
len(bank_accounts_ids)

135

In [129]:
#2loop over all companies and assign new bank accounts

##loop over all companies
#generate idn	bank_acc	iban	balance

 

#after you finish the loop, make sure you update the bank_accounts_ids dataset i.e. to add the current data there
# i.e. you can load the data again and concat and then write everything again

for index, row in companies.iterrows():
    
    country= row['country'] 
   
    country_code = countries.query("country==@country")['country_code'].iloc[0]
    #couple with a bank
    bank_company=  banks.query("country==@country").sample(n=1)  
    bank_name = bank_company.company.values[0] 
    bank_code =bank_company.bank_code.values[0]    
    bank_idn = bank_company.idn.values[0] 
    
    
    idn= unique_idn(nr_digits=10, search_list=bank_accounts_ids.idn.tolist()) 
    total_capacity = row['total_capacity']
    balance= total_capacity*1000000 # make it more realistic later?
    


    bank_acc= generate_new_bank_acc(country_code=country_code,bank_code=bank_code, location_code='' , acc_nr='')
    #now write to the df
    companies.loc[index, 'idn'] = idn
    companies.loc[index, 'iban'] =  bank_acc['iban'] 
    companies.loc[index, 'acc_nr'] = bank_acc['acc_nr']
    companies.loc[index, 'balance'] = balance
    
    #now write to bank_accounts_ids
    bank_accounts_ids_new_dict = {'idn':idn,  'iban':	bank_acc['iban'],

                                      'acc_nr':bank_acc['acc_nr']  ,
                                      'currency': bank_acc['currency'],
                                      'country_code': country_code,
                                      'country':country,
                                      'balance': balance ,
                                   'bank_name':  bank_name,
                                     'bank_idn':bank_idn}

    bank_accounts_ids_new  = pd.DataFrame([bank_accounts_ids_new_dict])
#         print ( bank_accounts_ids_new ) 
       
    bank_accounts_ids  = pd.concat([bank_accounts_ids,bank_accounts_ids_new  ], ignore_index=True)  
    
print(f"all {len (companies)} companies are assigned accounts" )    
   
# bank_accounts_ids

all 4563 companies are assigned accounts


In [93]:
len(bank_accounts_ids)  


4698

In [130]:

for index, row in people.iterrows():
#     idn=row['idn']
#     iban = row['iban'] 
    
#     if int(idn) not in   bank_accounts_ids.idn.tolist() and iban not in bank_accounts_ids.iban.tolist():

    country_code= row['country_code']
    country= row['country'] 
    balance = row['initial_balance'] 
    idn = row['idn'] 
    working = row['working'] 
    work_company=  row['work_company']
    #assign  a bank for that person
    bank_company=  banks.query("country==@country").sample(n=1)  
    bank_name = bank_company.company.values[0] 
    bank_code =bank_company.bank_code.values[0]   # bank['bank_code']
    bank_idn = bank_company.idn.values[0] 

    #make a bank account : 
    bank_acc= generate_new_bank_acc(country_code=country_code,bank_code=bank_code, location_code='' , acc_nr='')
    #now after we have the bank_acc 
    bank_acc['balance']= balance

   

    
    
    bank_accounts_ids_new_dict = {'idn':idn,
#   this is not neccessary(you will have to update it later, so skip for now)   	'bank_acc':	bank_acc,
                                  'iban': bank_acc['iban'],
                                  'acc_nr':bank_acc['acc_nr']  ,
                                  'currency': bank_acc['currency'],
                                  'country_code': country_code,
                                  'country':country,
                                  'balance': balance ,
                                 'bank_name': bank_name,
                                 'bank_idn':bank_idn }

#         print ( bank_accounts_ids_new_dict)      
    bank_accounts_ids_new  = pd.DataFrame([bank_accounts_ids_new_dict])
#         print ( bank_accounts_ids_new ) 

    bank_accounts_ids  = pd.concat([bank_accounts_ids,bank_accounts_ids_new  ], ignore_index=True)        
    
    
    #rewrite assigned companies: 
    id_housing_company=np.random.choice (companies.query("country==@country and high_level_category=='Housing'").idn.tolist())
    id_energy_company= np.random.choice  (companies.query("country==@country and high_level_category=='Energy'").idn.tolist())
    id_health_ins_company =  np.random.choice  (companies.query("country==@country and high_level_category=='Health Insurance'").idn.tolist())
    id_internet_company= np.random.choice  (companies.query("country==@country and high_level_category=='Internet'").idn.tolist())
    
    if working !='0':
        
        id_work_company=np.random.choice (companies.query("country==@country ").idn.tolist())
    else:
        
        id_work_company=  companies.query("company == @work_company").company.values[0]


    housing_company =      companies.query("idn==@id_housing_company").company.values[0]


    energy_company=      companies.query("idn==@id_energy_company").company.values[0]
    health_ins_company=   companies.query("idn==@id_health_ins_company").company.values[0]
    internet_company=    companies.query("idn==@id_internet_company").company.values[0]
    work_company=    companies.query("idn==@id_work_company").company.values[0]
    
    

      #now write to the df
     
    
    people.loc[index, 'bank_name'] = bank_name
    people.loc[index, 'bank_idn'] = bank_idn

    people.loc[index, 'id_housing_company'] = id_housing_company
    people.loc[index, 'id_energy_company'] = id_energy_company
    people.loc[index, 'id_health_ins_company'] = id_health_ins_company
    people.loc[index, 'id_internet_company'] = id_internet_company
    
    people.loc[index, 'housing_company'] = housing_company
    people.loc[index, 'energy_company'] = energy_company
    people.loc[index, 'health_ins_company'] = health_ins_company
    people.loc[index, 'internet_company'] = internet_company
    
    people.loc[index, 'work_company'] = work_company
    people.loc[index, 'id_work_company'] = id_work_company
 

print(f"all {len (people)} people are assigned accounts" )    


all 3000 people are assigned accounts


In [124]:
work_company ='Coffee & Co'
companies.query("company==@work_company").idn.values[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [114]:
for index, row in people.head().iterrows():
    work_company = row['work_company']
    print(companies.query("company==@work_company").idn.values[0])


1112078028
3788381289
2510308133
9661633013
0679651068


In [95]:
len(bank_accounts_ids)  

7698

In [96]:
people.sample(n=10)

,idn,first_name,last_name,gender,country,country_code,exchange_rate,male_prob,mean_age,age,education_level,education_title,profession,years_experience,working,source_income,salary_eur,salary,initial_balance_eur,initial_balance,hobby,spendingBehaviour,work_company,id_work_company,id_housing_company,id_energy_company,id_health_ins_company,id_internet_company,housing_company,energy_company,health_ins_company,internet_company,mean_monthly_rent_costs,mean_monthly_energy_costs,mean_monthly_internet_costs,mean_monthly_health_ins_costs,monthly_rent_costs,monthly_energy_costs,monthly_internet_costs,monthly_health_ins_costs,bank_name,bank_idn,mean_monthly_groceries_costs,monthly_groceries_costs
289,9529826979,Adrien,Baptiste,male,Belgium,BE,1.00,0.489,42.9,41,Postgraduate,Master of Environmental Studies,Sustainability Consultant,15,1,work,5034.0,5034.00,175789,175789.00,Painting,Middle-Class_Value_Seeker,Cora,2021962373,3179813896,2782071790,9492009291,0889451510,Immobel,ENGIE Belgium,AXA Belgium,Orange Belgium,1000,140,40,220,728.35,181.66,42.79,184.22,Belfius Bank,2910815252,320,290.32
642,9533285702,Bartosz,Tomasz,male,Poland,PL,4.50,0.482,44.1,47,Postgraduate,Master's in Finance,Senior Financial Analyst,21,1,work,2030.0,9135.00,46936,211212.00,Surfing,Upper-Class_Financially_Stable,Kawiarnia Literacka,9499048313,0975036585,2738484926,6303586234,9333006006,Polnord,Tauron Polska Energia,Compensa,T-Mobile Poland,3500,500,100,800,2504.21,563.69,94.95,1010.18,Santander Bank Polska,2709512416,1200,1117.51
1859,5161202842,Petar,Deyan,male,Bulgaria,BG,1.96,0.485,44.7,55,Postgraduate,Master of Computer Science,Software Engineer,33,1,work,1587.0,3110.52,80889,158542.44,Camping,Upper-Class_Financially_Stable,Albena Investment Holding,5789135793,6026587352,2177941638,0403904499,1037278413,Glavbolgarstroy,LUKOIL Bulgaria,Allianz Bulgaria,Max Telecom,800,160,40,200,975.20,146.22,44.41,200.79,DSK Bank,6637693295,350,396.52
1444,1788148315,Pierre,Daniel,male,Belgium,BE,1.00,0.489,42.9,37,Postgraduate,Master of Environmental Studies,Sustainability Consultant,13,1,work,4929.0,4929.00,141192,141192.00,Cycling,Middle-Class_Value_Seeker,Electrabel,7244697933,3881367581,1200056042,2309834376,0889451510,Xior Student Housing,Fluxys,NN Insurance,Orange Belgium,1000,140,40,220,865.29,133.50,38.90,210.29,BNP Paribas Fortis,3984421295,320,324.15
1545,7282332365,Lea,Luca,female,Germany,DE,1.00,0.489,46.7,63,Postgraduate,PhD in Physics,Astrophysicist,36,1,work,6229.0,6229.00,553719,553719.00,Gardening,Upper-Class_Financially_Stable,UniCredit Bank Germany,8855642102,0601136098,3037700916,9460116132,1910487227,LEG Immobilien AG,E.ON AG,AOK,Vodafone Germany,1100,160,35,280,1104.87,155.91,35.54,247.98,DZ Bank,6585396872,320,337.98
1398,7773713920,David,Vojtěch,male,Czech Republic,CZ,25.00,0.492,43.5,50,Postgraduate,Master of Economics,Economic Analyst,27,1,work,3059.0,76475.00,177161,4429025.00,Rock climbing,Upper-Class_Financially_Stable,Zabka Czech Republic,9365507548,0072821996,9360136276,2031525277,9059028513,Triple L Group,RWE Czech Republic,Allianz CZ,T-Mobile Czech Republic,20000,3000,600,4000,22138.31,3549.50,641.08,3583.93,Moneta Money Bank,3342827163,7500,8838.35
298,6183527002,Elisabeth,Daniel,female,Austria,AT,1.00,0.489,45.8,44,Postgraduate,PhD in Clinical Psychology,Clinical Psychologist,20,1,work,4545.0,4545.00,204279,204279.00,Knitting,Upper-Class_Financially_Stable,Generali Austria,7533213635,5358272407,2458853310,8810835091,2384425584,Raiffeisen Immobilien,OMV,Bupa Austria,UPC Austria,950,150,35,200,782.73,180.91,29.65,207.31,UniCredit Bank Austria,2625688558,300,360.93
1003,5715115066,Claire,Quentin,female,Belgium,BE,1.00,0.489,45.6,30,Undergraduate,Bachelor of Environmental Science,Environmental Consultant,6,1,work,4072.0,4072.00,59120,59120.00,Surfing,Middle-Class_Value_Seeker,Anheuser-Busch InBev,510719667,7452494200,1200056042,8353246997,0889451510,Besix,Fluxys,Baloise,Orange Belgium,1000,140,40,220,1076.50,194.73,32.70,261.53,Argenta,64987392

In [80]:
banks.query("idn== '1193394152' ")

,country,company,category,total_capacity,nr_employees,country_code,bank_code,idn,acc_nr,iban,balance
69,Ireland,Rabobank Ireland,Banks,2000,1800,IE,2900,1193394152,8200331542,IE1629008200331542,2000000000


In [97]:

companies.query("idn== '0889451510' ")

,country,company,high_level_category,total_capacity,nr_employees,idn,acc_nr,iban,balance,transaction_categories
193,Belgium,Orange Belgium,Internet,12000,6000,0889451510,2910230323,BE2793002910230323,12000000000,"['Web Hosting', 'Domain Registration', 'Online..."


In [72]:
'iban' in people.columns

False

In [131]:
bank_accounts_ids['idn'] = bank_accounts_ids['idn'] .astype(str)
banks['idn'] = banks['idn'] .astype(str)
companies['idn'] = companies['idn'] .astype(str)
people['idn'] = people['idn'] .astype(str)



In [85]:
bank_accounts_ids.query("idn ==  '1193394152'  ")

,idn,iban,acc_nr,currency,country,country_code,balance,bank_name,bank_idn
69,1193394152,IE1629008200331542,8200331542,EUR,Ireland,IE,2.000000e+09,Rabobank Ireland,1193394152


In [132]:
bank_accounts_ids.to_excel('bank_accounts_ids.xlsx', index=False)
banks.to_excel('banks.xlsx', index=False)
companies.to_excel('companies.xlsx', index=False)
people.to_excel('people.xlsx', index=False)

 

print('done')

done
